# lstm 生成模型  anna

可以参照官方文档  这个也是生成模型   
https://tensorflow.google.cn/tutorials/text/text_generation

## 1数据处理

In [1]:
import time
from collections import namedtuple

import numpy as np

In [2]:
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers

In [3]:
gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
cpus = tf.config.experimental.list_physical_devices(device_type='CPU')
print(gpus, cpus)

[] [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


In [4]:
with open('../data/anna.txt', 'r') as f:
    text=f.read()
print(len(text))
text=text[:int(len(text)/10)]
print(len(text))

1985223
198522


In [5]:
vocab = set(text)
vocab_to_int = {c: i for i, c in enumerate(vocab)}
int_to_vocab = dict(enumerate(vocab))
encoded = np.array([vocab_to_int[c] for c in text], dtype=np.int32)

In [6]:
text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

In [7]:
encoded[:100]

array([58, 70, 24, 63, 35, 48, 37, 49, 19, 59, 59, 59,  5, 24, 63, 63, 71,
       49, 18, 24, 73, 32,  3, 32, 48, 67, 49, 24, 37, 48, 49, 24,  3,  3,
       49, 24,  3, 32, 27, 48, 53, 49, 48, 33, 48, 37, 71, 49, 14, 34, 70,
       24, 63, 63, 71, 49, 18, 24, 73, 32,  3, 71, 49, 32, 67, 49, 14, 34,
       70, 24, 63, 63, 71, 49, 32, 34, 49, 32, 35, 67, 49, 54, 51, 34, 59,
       51, 24, 71, 72, 59, 59, 20, 33, 48, 37, 71, 35, 70, 32, 34],
      dtype=int32)

In [8]:
len(vocab)

75

In [9]:
print(len(text))
print(len(encoded))

encoded_train=encoded[1:]
print(len(encoded_train))
encoded_train=np.append(encoded_train,0)
print(len(encoded_train))
encoded_train[:100]

198522
198522
198521
198522


array([70, 24, 63, 35, 48, 37, 49, 19, 59, 59, 59,  5, 24, 63, 63, 71, 49,
       18, 24, 73, 32,  3, 32, 48, 67, 49, 24, 37, 48, 49, 24,  3,  3, 49,
       24,  3, 32, 27, 48, 53, 49, 48, 33, 48, 37, 71, 49, 14, 34, 70, 24,
       63, 63, 71, 49, 18, 24, 73, 32,  3, 71, 49, 32, 67, 49, 14, 34, 70,
       24, 63, 63, 71, 49, 32, 34, 49, 32, 35, 67, 49, 54, 51, 34, 59, 51,
       24, 71, 72, 59, 59, 20, 33, 48, 37, 71, 35, 70, 32, 34, 45])

## 2模型构建

In [17]:
batch_size = 64         # Sequences per batch
num_steps = 128
lstm_size = 768         # Size of hidden layers in LSTMs
learning_rate = 0.002    # Learning rate
keep_prob = 0.5         # Dropout keep probability
vocab_size=len(vocab)

In [18]:
vocab_size

75

In [19]:
int(len(encoded)/(batch_size*num_steps))*(batch_size*num_steps)

196608

In [20]:
def reshap_onehot_arr(arr,time_steps,cls_num):
    arr=arr[:int(len(encoded)/(batch_size*num_steps))*(batch_size*num_steps)]
    arr=np.reshape(arr,(-1,time_steps))
    arr=tf.keras.utils.to_categorical(arr,cls_num,dtype='int')
    print(arr.shape)
    return arr

train_x=reshap_onehot_arr(encoded,num_steps,vocab_size)
train_y=reshap_onehot_arr(encoded_train,num_steps,vocab_size)

(1536, 128, 75)
(1536, 128, 75)


 https://huhuhang.com/post/machine-learning/lstm-return-sequences-state  解释两个重要参数

In [21]:
def build_model(batch_size,stateful_flag,return_sequences_flag):
    model = keras.Sequential([
        layers.LSTM(lstm_size,
#                     dropout=keep_prob,
#                     recurrent_dropout=keep_prob,
                    return_sequences=return_sequences_flag,
                    stateful=stateful_flag,
                   batch_input_shape=[batch_size,None,vocab_size]),
        layers.Dense(vocab_size,activation="softmax")

    
    ])
    return model


model=build_model(batch_size=batch_size,stateful_flag=True,return_sequences_flag=True)

In [22]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate),
                 loss=keras.losses.CategoricalCrossentropy(),
                 metrics=['accuracy'])

In [23]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (64, None, 768)           2592768   
_________________________________________________________________
dense_1 (Dense)              (64, None, 75)            57675     
Total params: 2,650,443
Trainable params: 2,650,443
Non-trainable params: 0
_________________________________________________________________


In [24]:
import os
# 检查点保存至的目录
checkpoint_dir = './training_checkpoints/lstm_anna/'

# 检查点的文件名
checkpoint_prefix = os.path.join(checkpoint_dir, "lstm_anna_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [26]:
history = model.fit(train_x, train_y, batch_size=batch_size, epochs=2,callbacks=[checkpoint_callback])
# 将整个模型保存为HDF5文件
# model.save('./model/anna_keras_lstm_epochs2.h5')

Epoch 1/2
24/24 [==============================] - 24s 991ms/step - loss: 2.0102 - accuracy: 0.4244
Epoch 2/2
24/24 [==============================] - 26s 1s/step - loss: 1.9584 - accuracy: 0.4352


In [27]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/lstm_anna/lstm_anna_2'

In [28]:
model_pre =build_model(batch_size=1,stateful_flag=True,return_sequences_flag=False)
model_pre.load_weights(tf.train.latest_checkpoint(checkpoint_dir)) #只加载最后一次的weight
model_pre.build(tf.TensorShape([1,None,vocab_size]))
model_pre.summary()


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (1, 768)                  2592768   
_________________________________________________________________
dense_2 (Dense)              (1, 75)                   57675     
Total params: 2,650,443
Trainable params: 2,650,443
Non-trainable params: 0
_________________________________________________________________


In [29]:
a=np.array([1,2,3])
a=a.reshape([1,-1,1])
print(a)
a=tf.keras.utils.to_categorical(a,vocab_size,dtype='float32')
a

[[[1]
  [2]
  [3]]]


array([[[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.,

In [30]:
test_predict_res = model_pre(a)
print(test_predict_res)
sum(test_predict_res.numpy().flatten())

tf.Tensor(
[[0.00895433 0.00738238 0.00769131 0.02747618 0.00730457 0.01471999
  0.01369404 0.00830105 0.00702767 0.00774915 0.01050662 0.01182116
  0.0123561  0.00773084 0.02194421 0.00768127 0.00552484 0.00807873
  0.01350781 0.00748674 0.01320702 0.00777904 0.00677212 0.00880008
  0.01791713 0.01541926 0.01987185 0.00969178 0.00524338 0.01209897
  0.00723721 0.01433095 0.02660017 0.00817015 0.00758837 0.0080409
  0.00920677 0.0126022  0.00725214 0.01025167 0.01662128 0.00498053
  0.00584034 0.02519404 0.00768138 0.0063764  0.01215357 0.0079821
  0.03001572 0.036899   0.00911816 0.01187509 0.00731351 0.01568682
  0.02076812 0.0088321  0.01563297 0.01285512 0.00996464 0.09136713
  0.01781917 0.00736531 0.01339475 0.00621431 0.0073671  0.01313248
  0.00719469 0.00862644 0.0073649  0.01096334 0.00678743 0.03276003
  0.03979056 0.00860991 0.01043132]], shape=(1, 75), dtype=float32)


0.9999999185092747

In [31]:
pre_distrubtion = test_predict_res.numpy().flatten()
print(max(pre_distrubtion))
print(tf.argmax(pre_distrubtion))
np.random.choice(np.array(range(pre_distrubtion.shape[0])), 20, replace=True, p=pre_distrubtion)


0.09136713
tf.Tensor(59, shape=(), dtype=int64)


array([46, 31, 11, 59, 58, 49, 64, 59, 71, 21, 70, 14, 72, 23, 65, 72, 31,
       37, 65, 20])

In [32]:
def generate_text(model, start_string):
    # 评估步骤（用学习过的模型生成文本）

    # 要生成的字符个数
    num_generate = 100

    # 将起始字符串转换为数字（向量化）
    input_eval = np.array([vocab_to_int[s] for s in start_string])
    print(input_eval)
    input_eval = input_eval.reshape([1,-1,1])
    input_eval=tf.keras.utils.to_categorical(input_eval,vocab_size,dtype='float32')
    print("input_eval",input_eval.shape)

    # 空字符串用于存储结果
    text_generated = []

    # 低温度会生成更可预测的文本
    # 较高温度会生成更令人惊讶的文本
    # 可以通过试验以找到最好的设定
    temperature = 1.0

    # 这里批大小为 1
    model.reset_states()
    for i in range(num_generate):
        
        predictions = model(input_eval)
        print("predictions",predictions.shape)        
#         predicted_id = tf.argmax(predictions.numpy().flatten()).numpy()
#         print("predicted_id",predicted_id)
        predictions_distr = predictions.numpy().flatten()
        index_list = np.array(range(predictions_distr.shape[0]))
        predicted_id = np.random.choice(index_list, 1, replace=True, p=predictions_distr)
        pre_char= int_to_vocab[list(predicted_id)[0]]
        print("predicted_char",pre_char)
        text_generated.append(pre_char)
        
        input_eval = np.array([vocab_to_int[s] for s in pre_char])
        input_eval = input_eval.reshape([1,-1,1])
        input_eval=tf.keras.utils.to_categorical(input_eval,vocab_size,dtype='float32')

    return (start_string + ''.join(text_generated))

generate_text(model_pre,"am")

[24 73]
input_eval (1, 2, 75)
predictions (1, 75)
predicted_char r
predictions (1, 75)
predicted_char .
predictions (1, 75)
predicted_char z
predictions (1, 75)
predicted_char "
predictions (1, 75)
predicted_char 

predictions (1, 75)
predicted_char d
predictions (1, 75)
predicted_char "
predictions (1, 75)
predicted_char W
predictions (1, 75)
predicted_char h
predictions (1, 75)
predicted_char l
predictions (1, 75)
predicted_char l
predictions (1, 75)
predicted_char  
predictions (1, 75)
predicted_char w
predictions (1, 75)
predicted_char o
predictions (1, 75)
predicted_char o
predictions (1, 75)
predicted_char d
predictions (1, 75)
predicted_char ?
predictions (1, 75)
predicted_char "
predictions (1, 75)
predicted_char 

predictions (1, 75)
predicted_char 

predictions (1, 75)
predicted_char "
predictions (1, 75)
predicted_char S
predictions (1, 75)
predicted_char h
predictions (1, 75)
predicted_char e
predictions (1, 75)
predicted_char  
predictions (1, 75)
predicted_char e
predicti

'amr.z"\nd"Whll wood?"\n\n"She eallich, but heranted sis? You ther becine of o he paskedent ham verrombeg '